In [1]:
from langchain.memory import ConversationBufferMemory
from util import SetupIndex
from prompts import rag_prompt_custom, contextualize_q_prompt, STANDALONE_QUESTION_PROMPT
from engine import Engine
from langchain.schema import AIMessage, HumanMessage


selected_model = "qwen2.5:0.5b"
embedding_model = 'nomic-embed-text'

model = Engine(selected_model, embedding_model)
index = SetupIndex(model.embeddings)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

retriever = index.create_retriever()

chat_history = list()

Loading data from the disk ...
Retriever loaded


C:\Users\ACER\AppData\Local\Temp\ipykernel_18668\940724847.py:14: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [3]:
from langchain_core.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    MessagesPlaceholder,
    ChatPromptTemplate)


system_prompt = "You are an AI assistant specialized in answering user queries accurately using retrieved documents."

# Chat prompt template with a placeholder for chat history
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),  # Dynamically update chat history
        ("human", "{input}"),
    ]
)



# system_rephrase_prompt  =  """Given the following conversation and a follow up question, 
# rephrase the follow up question to be a standalone question, in its original language, 
# that can be used to query a FAISS index. This query will be used to retrieve documents with additional context.

# Let me share a couple examples.

# If you do not see any chat history, you MUST return the "Follow Up Input" as is:
# ```
# Chat History:
# Follow Up Input: How is Lawrence doing?
# Standalone Question:
# How is Lawrence doing?
# ```

# If this is the second question onwards, you should properly rephrase the question like this:
# ```
# Chat History:
# Human: How is Lawrence doing?
# AI: 
# Lawrence is injured and out for the season.
# Follow Up Input: What was his injury?
# Standalone Question:
# What was Lawrence's injury?

# ```
# Now, with those examples, here is the actual chat history and input question.
# Chat History:

# """

# # Chat prompt template with a placeholder for chat history
# query_rehrase_prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_rephrase_prompt),
#         MessagesPlaceholder("chat_history"),  # Dynamically update chat history
#         ("human", "{input}"),
#     ]
# )

# from prompts import rag_prompt_custom, contextualize_q_prompt, STANDALONE_QUESTION_PROMPT

# pr = STANDALONE_QUESTION_PROMPT.format_prompt(chat_history=chat_history, question=input_query)

In [2]:
def chat(input_query):

    retrieved_docs  = retriever.invoke(input_query)
    context         = index.process_text(retrieved_docs)

    prompt    = rag_prompt_custom.invoke({"context": context, "question":input_query})
    response  = model.llm_engine.invoke(prompt)

    return response

In [3]:
count = 0

while count < 2:

    input_query  = input("human: ")

    response = chat(input_query)
    
    # chat_history.append(HumanMessage(content=input_query))
    # chat_history.append(AIMessage(content=response.content))

    # print(qa_prompt.format_prompt(chat_history=chat_history, input=input_query))
    # print("Assistant : ", response.content)

    memory.save_context({"input": input_query}, {"output": response.content})

    count += 1

    


In [4]:
memory.load_memory_variables({})["chat_history"]

[HumanMessage(content='what is fitness', additional_kwargs={}, response_metadata={}),
 AIMessage(content="Fitness refers to a state of physical health and well-being achieved through regular exercise, healthy eating habits, and maintaining an active lifestyle. It encompasses various activities such as running, cycling, swimming, yoga, weightlifting, and more. Fitness supplements are essential for athletes, fitness enthusiasts, and individuals looking to improve their performance in sports or daily activities.\n\nSupplementary drinks play a crucial role in enhancing physical abilities and boosting energy levels during workouts. They often contain high-quality ingredients like protein powder, electrolytes, and vitamins that help maintain an optimal state of hydration and nutrition.\n\nEducational programs and marketing strategies are employed to promote the responsible and informed use of fitness supplements among enthusiasts, emphasizing their positive impact on overall health and well-

In [ ]:
count = 0

while count < 2:

    input_query  = input("human: ")

    retrieved_docs  = retriever.invoke(input_query)
    context         = index.process_text(retrieved_docs)

    prompt    = rag_prompt_custom.invoke({"context": context, "question":input_query})
    response  = model.llm_engine.invoke(prompt)
    
    chat_history.append(HumanMessage(content=input_query))
    chat_history.append(AIMessage(content=response.content))

    # print(qa_prompt.format_prompt(chat_history=chat_history, input=input_query))

    count += 1

    
    # print("Assistant : ", response.content)


messages=[SystemMessage(content='You are an AI assistant specialized in answering user queries accurately using retrieved documents.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is the fitness', additional_kwargs={}, response_metadata={}), AIMessage(content="Fitness supplements are commonly used to enhance performance in various sports and health settings. They cater to both competitive athletes and non-competitive enthusiasts who seek a balanced approach to health and fitness. The incorporation of the framework represents a novel perspective on studying health-related behavior, offering a more nuanced understanding of psychological, belief-based, and motivational factors.\n\nTo better understand these supplements, it's essential to consider their potential benefits for individuals with different goals in mind, including increasing muscle mass, enhancing performance in fitness activities, meeting nutritional needs, and supporting the maintenance or initiati

In [7]:
# print(query_rehrase_prompt.format_prompt(chat_history=chat_history, input=input_query).to_string())

In [1]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

qa_system_prompt = "You are an AI assistant that helps with Python programming."

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),  # System role with fixed instructions
        MessagesPlaceholder("chat_history"),  # Placeholder for past conversation
        ("human", "{input}")  # User's question goes here dynamically
    ]
)

# Example input
chat_history = [
    ("human", "What is Python?"),
    ("ai", "Python is a programming language.")
]

user_input = "How do I create a list in Python?"

formatted_prompt = qa_prompt.format(chat_history=chat_history, input=user_input)

print(formatted_prompt)


System: You are an AI assistant that helps with Python programming.
Human: What is Python?
AI: Python is a programming language.
Human: How do I create a list in Python?
